# Prompt

You are tasked with generating Python scripts that perform various BGP analysis tasks using the pybgpstream library. Please adhere to the following guidelines when writing the code:

- Main Loop Processing:
Do not use any filter attributes like stream.add_filter() or set filter parameters when initializing BGPStream.
All filtering and processing should occur within the main loop where you iterate over records and elements.

- Script Structure:
Start by importing necessary libraries, including pybgpstream and any others required for the task (e.g., datetime, collections).
Define a main function or functions that encapsulate the core logic.
Include a __main__ block or a usage example to demonstrate how to run the script.

- Key Processing Guidelines:
* Time Format: Define the time range as strings in the following format: from_time = "YYYY-MM-DD HH:MM:SS"
until_time = "YYYY-MM-DD HH:MM:SS"

* Stream Initialization: Use these time parameters during BGPStream initialization:
stream = pybgpstream.BGPStream(
    from_time=from_time,
    until_time=until_time,
    record_type="updates",
    collectors=collectors
)

* Iterating Over Records and Elements:
for rec in stream.records(): for elem in rec: Processing logic goes here

* Accessing Element Attributes:
Timestamp: elem_time = datetime.utcfromtimestamp(elem.time)

Element Type (Announcement or Withdrawal): elem_type = elem.type 'A' for announcements, 'W' for withdrawals

Fields Dictionary: fields = elem.fields

Prefix: prefix = fields.get("prefix") if prefix is None: continue

AS Path: as_path_str = fields.get('as-path', "") as_path = as_path_str.split()

Peer ASN and Collector: peer_asn = elem.peer_asn collector = rec.collector

Communities: communities = fields.get('communities', [])

* Filtering Logic Within the Loop:
Filtering for a Specific ASN in AS Path: target_asn = '64500' if target_asn not in as_path: continue

Filtering for Specific Prefixes: target_prefixes = ['192.0.2.0/24', '198.51.100.0/24'] if prefix not in target_prefixes: continue

* Processing Key Values and Attributes:
Counting Announcements and Withdrawals: if elem_type == 'A': announcements[prefix] += 1 elif elem_type == 'W': withdrawals[prefix] += 1

Detecting AS Path Changes: if prefix in prefix_as_paths: if as_path != prefix_as_paths[prefix]: # AS path has changed prefix_as_paths[prefix] = as_path else: prefix_as_paths[prefix] = as_path

Analyzing Community Attributes: for community in communities: community_str = f"{community[0]}:{community[1]}" community_counts[community_str] += 1

Calculating Statistics (e.g., Average MED): med = fields.get('med') if med is not None: try: med_values.append(int(med)) except ValueError: pass

* Detecting Hijacks: Compare the observed origin AS with the expected origin AS for target prefixes:
expected_origins = {'192.0.2.0/24': '64500', '198.51.100.0/24': '64501'}
if prefix in expected_origins:
    observed_origin = as_path[-1] if as_path else None
    expected_origin = expected_origins[prefix]
    if observed_origin != expected_origin:
        # Potential hijack detected
        print(f"Possible hijack detected for {prefix}: expected {expected_origin}, observed {observed_origin}")

* Detecting Outages:
* Monitor for withdrawals of prefixes without re-announcements:
Keep track of withdrawn prefixes and their timestamps
if elem_type == 'W':
    withdrawals[prefix] = elem_time
elif elem_type == 'A':
    # Remove from withdrawals if re-announced
    if prefix in withdrawals:
        del withdrawals[prefix]
Check if prefix remains withdrawn for a certain period (e.g., 30 minutes)
for prefix, withdrawal_time in list(withdrawals.items()):
    if elem_time - withdrawal_time > timedelta(minutes=30):
        # Outage detected for prefix
        print(f"Outage detected for {prefix} starting at {withdrawal_time}")
        del withdrawals[prefix]

* Detecting MOAS (Multiple Origin AS) Conflicts: Monitor prefixes announced by multiple origin ASNs
origin_asn = as_path[-1] if as_path else None
if origin_asn:
    if prefix not in prefix_origins:
        prefix_origins[prefix] = set()
    prefix_origins[prefix].add(origin_asn)
    if len(prefix_origins[prefix]) > 1:
        # MOAS conflict detected
        origins = ', '.join(prefix_origins[prefix])
        print(f"MOAS conflict for {prefix}: announced by ASNs {origins}")

* Analyzing AS Path Prepending: Detect AS path prepending by identifying consecutive repeated ASNs in the AS path:
last_asn = None
consecutive_count = 1
for asn in as_path:
    if asn == last_asn:
        consecutive_count += 1
    else:
        if consecutive_count > 1:
            prepending_counts[last_asn] += consecutive_count - 1
        consecutive_count = 1
    last_asn = asn
Check for prepending at the end of the path
if consecutive_count > 1 and last_asn:
    prepending_counts[last_asn] += consecutive_count - 1
    
* Handling IP Addresses and Prefixes:
Validating and Parsing IP Prefixes: import ipaddress try: network = ipaddress.ip_network(prefix) except ValueError: continue

Here is your tasks:

###  Track AS-level trends over period from 13pm to 13:10 pm for AS3356. Summarize metrics like announcements and withdrawals with print statements

In [ ]:
import pybgpstream
from datetime import datetime
from collections import defaultdict
import ipaddress

def analyze_bgp_messages():
    # Define the time range
    from_time = "2024-10-28 12:00:00"
    until_time = "2024-10-29 12:00:00"

    # Specify the collector and target ASN
    collectors = ["rrc00"]
    target_asn = "3356"

    # Initialize BGPStream
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    # Initialize counters and data structures
    announcements = defaultdict(int)
    withdrawals = defaultdict(int)
    prefix_as_paths = {}
    community_counts = defaultdict(int)
    med_values = []

    # Iterate over records and elements
    for rec in stream.records():
        for elem in rec:
            # Process the element
            elem_time = datetime.utcfromtimestamp(elem.time)
            elem_type = elem.type  # 'A' for announcements, 'W' for withdrawals
            fields = elem.fields

            # Get the prefix
            prefix = fields.get("prefix")
            if prefix is None:
                continue  # Skip if prefix is not present

            # Get the AS path
            as_path_str = fields.get('as-path', "")
            as_path = as_path_str.split()

            # Check if the target ASN is in the AS path
            if target_asn not in as_path:
                continue  # Skip if target ASN is not in the AS path

            # Get peer ASN and collector
            peer_asn = elem.peer_asn
            collector = rec.collector

            # Count announcements and withdrawals
            if elem_type == 'A':
                announcements[prefix] += 1
            elif elem_type == 'W':
                withdrawals[prefix] += 1

            # Detect AS path changes
            if prefix in prefix_as_paths:
                if as_path != prefix_as_paths[prefix]:  # AS path has changed
                    prefix_as_paths[prefix] = as_path
            else:
                prefix_as_paths[prefix] = as_path

            # Analyze community attributes
            communities = fields.get('communities', [])
            for community in communities:
                community_str = f"{community[0]}:{community[1]}"
                community_counts[community_str] += 1

            # Calculate statistics (e.g., Average MED)
            med = fields.get('med')
            if med is not None:
                try:
                    med_values.append(int(med))
                except ValueError:
                    pass  # Ignore invalid MED values

    # Calculate average MED if values exist
    if med_values:
        med_avg = sum(med_values) / len(med_values)
        print(f"Average MED: {med_avg:.2f}")
    # Print the results
    print("Announcements:", dict(announcements))
    print("Withdrawals:", dict(withdrawals))
    print("Communities Count:", dict(community_counts))

if __name__ == "__main__":
    analyze_bgp_messages()

# ~35 minutes

In [ ]:
import pybgpstream
from datetime import datetime
from collections import defaultdict
import ipaddress

def analyze_bgp_messages():
    # Define the time range
    from_time = "2024-10-28 12:00:00"
    until_time = "2024-10-29 12:00:00"

    # Specify the collector and target ASN
    collectors = ["rrc00"]
    target_asn = "3356"

    # Define filter to limit processing to relevant records only
    filter_str = f"collector rrc00 and type updates and ipversion 4 and path _{target_asn}_"

    # Initialize BGPStream with the filter
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        filter=filter_str  # Apply the filter string
    )

    # Initialize counters and data structures
    announcements = defaultdict(int)
    withdrawals = defaultdict(int)
    prefix_as_paths = {}
    community_counts = defaultdict(int)
    med_values = []

    # Iterate over records and elements
    for rec in stream.records():
        for elem in rec:
            # Process the element
            elem_time = datetime.utcfromtimestamp(elem.time)
            elem_type = elem.type  # 'A' for announcements, 'W' for withdrawals
            fields = elem.fields

            # Get the prefix
            prefix = fields.get("prefix")
            if prefix is None:
                continue  # Skip if prefix is not present

            # Get the AS path
            as_path_str = fields.get('as-path', "")
            as_path = as_path_str.split()

            # Check if the target ASN is in the AS path (double-checking as a safeguard)
            if target_asn not in as_path:
                continue  # Skip if target ASN is not in the AS path

            # Count announcements and withdrawals
            if elem_type == 'A':
                announcements[prefix] += 1
            elif elem_type == 'W':
                withdrawals[prefix] += 1

            # Detect AS path changes
            if prefix in prefix_as_paths:
                if as_path != prefix_as_paths[prefix]:  # AS path has changed
                    prefix_as_paths[prefix] = as_path
            else:
                prefix_as_paths[prefix] = as_path

            # Analyze community attributes
            communities = fields.get('communities', [])
            for community in communities:
                community_str = f"{community[0]}:{community[1]}"
                community_counts[community_str] += 1

            # Calculate statistics (e.g., Average MED)
            med = fields.get('med')
            if med is not None:
                try:
                    med_values.append(int(med))
                except ValueError:
                    pass  # Ignore invalid MED values

    # Calculate average MED if values exist
    if med_values:
        med_avg = sum(med_values) / len(med_values)
        print(f"Average MED: {med_avg:.2f}")

    # Print the results
    print("Announcements:", dict(announcements))
    print("Withdrawals:", dict(withdrawals))
    print("Communities Count:", dict(community_counts))

if __name__ == "__main__":
    analyze_bgp_messages()

### detect hijacking from as3356 from oct 28 13pm to 13:30 pm 2024

In [1]:
import pybgpstream
from datetime import datetime, timedelta
import collections
import ipaddress

def detect_hijacking(from_time, until_time, target_asn):
    # Initialize BGPStream with the specified time range and record type
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates"
    )

    # Initialize dictionaries to track potential hijacks
    observed_origins = {}
    expected_origins = {'192.0.2.0/24': '3356', '198.51.100.0/24': '64501'}  # Example prefixes

    # Iterate over BGP records and their elements
    for rec in stream.records():
        for elem in rec:
            # Access element attributes
            elem_time = datetime.utcfromtimestamp(elem.time)
            elem_type = elem.type  # 'A' for announcements, 'W' for withdrawals
            fields = elem.fields

            # Get prefix and skip if not present
            prefix = fields.get("prefix")
            if prefix is None:
                continue

            # Get AS path and peer ASN
            as_path_str = fields.get('as-path', "")
            as_path = as_path_str.split()
            peer_asn = elem.peer_asn
            
            # Detect hijacks by comparing observed and expected origins
            if prefix in expected_origins:
                observed_origin = as_path[-1] if as_path else None
                expected_origin = expected_origins[prefix]
                
                if observed_origin != expected_origin:
                    print(f"Possible hijack detected for {prefix}: expected {expected_origin}, observed {observed_origin} from AS {peer_asn} at {elem_time}")

if __name__ == "__main__":
    # Define the time range for analysis
    from_time = "2024-10-28 13:00:00"
    until_time = "2024-10-28 13:30:00"
    target_asn = '3356'

    # Run the hijacking detection function
    detect_hijacking(from_time, until_time, target_asn)